In [3]:
# ==============================================================================
# YoLearn.ai Autonomous AI Tutor Orchestrator
# ==============================================================================
# This system automatically understands what students need and calls the right
# educational tools to help them learn. It's like having a smart assistant that
# knows when you need practice problems, explanations, or study notes.

# STEP 1: Setup and Configuration
# ==============================================================================

# Set up your Hugging Face token to access the AI model
import os
os.environ['HF_TOKEN'] = 'PASTE YOUR CODE HERE '  # Replace with your actual token

# Import all the libraries we need
import nest_asyncio
nest_asyncio.apply()  # This helps the code run smoothly in Jupyter/Colab

from openai import OpenAI
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional, List, Dict, Any
from datetime import datetime
from fastapi.testclient import TestClient

# Connect to DeepSeek-R1 AI model (this is our "brain" that understands students)
client = OpenAI(base_url="https://router.huggingface.co/v1", api_key=os.environ['HF_TOKEN'])

# STEP 2: Define Data Structures
# ==============================================================================
# These are like forms that define what information we need to collect

# What the student sends us (their question/request + who they are)
class OrchestratorRequest(BaseModel):
    user_input: str          # What the student typed (e.g., "I need help with math")
    user_id: str            # Student's ID to track their learning
    session_id: Optional[str] = None  # Optional: to continue previous conversations

# What we send back to the student
class OrchestratorResponse(BaseModel):
    success: bool           # Did everything work correctly?
    response: str          # The actual helpful response for the student
    intent: str            # What did the student want? (practice, explanation, etc.)
    topic: str             # What subject/topic? (math, science, etc.)
    emotional_state: str   # How is the student feeling? (frustrated, confident, etc.)
    suggestions: List[str] = []  # Follow-up suggestions for the student

# Information about which educational tool we used and how
class ToolExecution(BaseModel):
    tool_name: str                    # Which tool did we use? (quiz_generator, etc.)
    request_params: Dict[str, Any]    # What settings did we use for the tool?
    raw_tool_response: Any            # Raw output from the educational tool
    formatted_response: str           # Cleaned up response for the student

# Complete response with all the details (extends OrchestratorResponse)
class FullOrchestratorResponse(OrchestratorResponse):
    tool_execution: ToolExecution     # Details about which tool was used
    session_id: str                   # Session ID for tracking conversations
    next_actions: List[str]           # What can the student do next?

# STEP 3: Create the Web Application
# ==============================================================================

# Create our web application that can receive and respond to student requests
app = FastAPI(title="YoLearn AI Orchestrator", version="1.0")

# STEP 4: Educational Tool Configurations
# ==============================================================================
# This defines what educational tools we have and their default settings

TOOL_SCHEMAS = {
    # Quiz Generator: Creates practice questions for students
    "quiz_generator": {
        "defaults": {
            "difficulty": "beginner",      # Start with easy questions
            "question_type": "practice",   # Practice-style questions
            "num_questions": 5             # Generate 5 questions by default
        }
    },
    # Concept Explainer: Explains difficult concepts in simple terms
    "concept_explainer": {
        "defaults": {
            "desired_depth": "basic"       # Start with basic explanations
        }
    },
    # Note Maker: Creates organized study notes
    "note_maker": {
        "defaults": {
            "note_taking_style": "structured",  # Well-organized notes
            "include_examples": True             # Include helpful examples
        }
    },
    # Flashcard Generator: Creates digital flashcards for memorization
    "flashcard_generator": {
        "defaults": {
            "count": 5,                    # Make 5 flashcards by default
            "difficulty": "medium",        # Medium difficulty level
            "include_examples": True       # Include examples on cards
        }
    }
}

# STEP 5: Memory System
# ==============================================================================
# This keeps track of each student's conversation history

SESSIONS = {}  # Dictionary to store all student conversations

# STEP 6: AI Brain Functions
# ==============================================================================
# These functions help the AI understand what students want

def extract_context(user_input: str) -> Dict[str,str]:
    """
    CONTEXT ANALYZER: This is like the AI's "listening ear"
    It reads what the student wrote and figures out:
    1. What do they want? (intent)
    2. What subject? (topic)
    3. How are they feeling? (emotional_state)
    """
    # Ask the AI to analyze the student's message
    prompt = """Extract JSON with keys: intent, topic, emotional_state from student message. Respond ONLY in JSON."""

    try:
        # Send the student's message to DeepSeek-R1 AI for analysis
        completion = client.chat.completions.create(
            model='deepseek-ai/DeepSeek-R1',
            messages=[{"role": "system", "content": prompt}, {"role": "user", "content": user_input}],
            temperature=0.7,     # How creative should the AI be? (0.7 is moderately creative)
            max_tokens=200       # Maximum length of AI response
        )

        # Extract the AI's response and find the JSON part
        resp = completion.choices[0].message.content.strip()
        import re
        json_part = re.search(r"\{.*\}", resp, flags=re.DOTALL).group()
        return eval(json_part)  # Convert JSON text to Python dictionary

    except:
        # If something goes wrong, provide a reasonable default response
        return {"intent":"request_practice_problems", "topic":"calculus_derivatives", "emotional_state":"frustrated"}

def pick_tool_from_intent(intent: str) -> str:
    """
    TOOL SELECTOR: Like a smart librarian who knows which tool to recommend
    Based on what the student wants, pick the best educational tool
    """
    il = intent.lower()  # Convert to lowercase for easier matching

    # If student wants practice, give them a quiz generator
    if "practice" in il or "quiz" in il or "problems" in il:
        return "quiz_generator"

    # If student wants notes or summary, give them note maker
    if "note" in il or "summary" in il:
        return "note_maker"

    # If student wants explanation, give them concept explainer
    if "explain" in il or "explanation" in il:
        return "concept_explainer"

    # Default: if we're not sure, give them quiz generator (most common need)
    return "quiz_generator"

def extract_tool_params(intent: str, topic: str, emotion: str, user_info=None, chat_history=None):
    """
    PARAMETER EXTRACTOR: Like a smart form-filler
    Takes what we know about the student and creates the perfect settings for the educational tool
    """
    # Step 1: Figure out which tool we need
    tool = pick_tool_from_intent(intent)

    # Step 2: Start with default settings for this tool
    defaults = TOOL_SCHEMAS[tool]["defaults"]
    params = dict(defaults)  # Make a copy of the defaults

    # Step 3: Extract subject from topic (e.g., "calculus derivatives" -> "calculus")
    subject = topic.split()[0] if topic else "general"

    # Step 4: EMOTIONAL INTELLIGENCE - Adjust difficulty based on how student feels
    diff = params.get("difficulty", "medium")
    if emotion in ["confused", "anxious", "frustrated"]:
        diff = "easy"     # If student is struggling, make it easier
    elif emotion == "confident":
        diff = "hard"     # If student is confident, challenge them more

    # Step 5: Fill in tool-specific parameters
    if tool == "quiz_generator":
        # For quiz generator: set topic, subject, difficulty, question type, and number
        params.update({
            "topic": topic,
            "subject": subject,
            "difficulty": diff,
            "question_type": params.get("question_type"),
            "num_questions": params.get("num_questions")
        })
    elif tool == "concept_explainer":
        # For concept explainer: set what to explain and how deep to go
        params.update({
            "user_info": user_info,
            "chat_history": chat_history,
            "concept_to_explain": topic,
            "current_topic": subject,
            "desired_depth": "basic" if diff == "easy" else "intermediate"
        })

    # Return both the tool name and its configured parameters
    return tool, params

async def call_tool(tool_name: str, params: Dict[str,Any]) -> ToolExecution:
    """
    TOOL EXECUTOR: Like a worker who actually does the educational task
    Takes the tool name and settings, then creates the educational content
    """
    # Create appropriate response based on which tool was selected
    if tool_name == "quiz_generator":
        # Generate a message about creating quiz questions
        raw = f"Generated {params['num_questions']} {params['difficulty']} questions on {params['topic']}"
    else:
        # For other tools, generate explanation message
        raw = f"Explained concept {params['topic']} at {params['difficulty']} depth"

    # Package up all the information about what we did
    return ToolExecution(
        tool_name=tool_name,           # Which tool we used
        request_params=params,         # Settings we used
        raw_tool_response=raw,         # What the tool produced
        formatted_response=raw         # Cleaned up version for the student
    )

def generate_suggestions(intent, emotion):
    """
    SUGGESTION GENERATOR: Like a helpful tutor who suggests next steps
    Based on what the student wanted and how they're feeling, suggest helpful follow-up actions
    """
    suggestions = []

    # If student wanted practice, suggest related study activities
    if "practice" in intent or "problems" in intent:
        suggestions.extend([
            "Generate flashcards for practice",      # Help with memorization
            "Provide concise notes summary",         # Get organized notes
            "Ask for detailed concept explanation"   # Understand the theory
        ])

    # If student is struggling emotionally, suggest breaking things down
    if emotion in ["confused", "anxious", "frustrated"]:
        suggestions.append("Break content into simpler parts")

    # Only return the first 3 suggestions to avoid overwhelming the student
    return suggestions[:3]

# STEP 7: Session Management Functions
# ==============================================================================
# These help us remember conversations with each student

def init_session(user_id: str, session_id: Optional[str] = None) -> str:
    """
    SESSION CREATOR: Like creating a new notebook for each student
    Sets up a space to remember this student's conversation
    """
    # If no session ID provided, create a unique one using user ID and current time
    if not session_id:
        session_id = f"{user_id}_{int(datetime.now().timestamp())}"

    # If this is a new session, create empty history and profile
    if session_id not in SESSIONS:
        SESSIONS[session_id] = {"history": [], "profile": {}}

    return session_id

def _default_user():
    """
    DEFAULT USER PROFILE: Creates a standard student profile
    This is used when we don't have specific information about the student
    """
    return {
        "user_id": "student_demo",
        "name": "Demo Student",
        "grade_level": "10",
        "learning_style_summary": "Structured learner",      # How they like to learn
        "emotional_state_summary": "Engaged",                # How they usually feel
        "mastery_level_summary": "Level 5: Developing"       # Their skill level
    }

# STEP 8: Web API Endpoints
# ==============================================================================
# These are like doors that students can knock on to get help

@app.get("/health")
def health_check():
    """
    HEALTH CHECK: Like checking if the system is awake and working
    Returns a simple message to confirm everything is running
    """
    return {"status": "healthy", "model": "deepseek-ai/DeepSeek-R1"}

@app.post("/api/orchestrate_full", response_model=FullOrchestratorResponse)
async def orchestrate_full(req: OrchestratorRequest):
    """
    MAIN ORCHESTRATOR: This is the brain of the whole system!
    Like a conductor of an orchestra, it coordinates all the different parts:
    1. Understands what the student needs
    2. Picks the right educational tool
    3. Configures the tool perfectly for this student
    4. Gets the educational content
    5. Adds helpful suggestions
    6. Sends everything back to the student
    """

    # Step 1: Set up session tracking for this student
    session_id = init_session(req.user_id, req.session_id)

    # Step 2: Use AI to understand what the student wants
    ctx = extract_context(req.user_input)
    intent, topic, emotion = ctx["intent"], ctx["topic"], ctx["emotional_state"]

    # Step 3: Get student profile and conversation history
    user_profile = _default_user()                           # Get default student info
    chat_history = SESSIONS[session_id]["history"]          # Get previous conversations

    # Step 4: Figure out which tool to use and how to configure it
    tool_name, tool_params = extract_tool_params(intent, topic, emotion, user_profile, chat_history)

    # Step 5: Actually use the educational tool to create content
    execution = await call_tool(tool_name, tool_params)

    # Step 6: Generate helpful suggestions for what to do next
    suggestions = generate_suggestions(intent, emotion)

    # Step 7: Package everything up and send it back to the student
    return FullOrchestratorResponse(
        success=True,                                    # Everything worked!
        response=execution.formatted_response,           # The main answer for the student
        intent=intent,                                   # What they wanted
        topic=topic,                                     # What subject
        emotional_state=emotion,                         # How they're feeling
        suggestions=suggestions,                         # Helpful next steps
        tool_execution=execution,                        # Details about what tool was used
        session_id=session_id,                          # Session ID for next time
        next_actions=["Review flashcards", "Request notes", "Get explanation"]  # More options
    )

# STEP 9: Test the System
# ==============================================================================
# Let's try it out to make sure everything works!

if __name__ == "__main__":
    # Create a test client to simulate a student using the system
    tc = TestClient(app)

    # Simulate a student asking for help (this is like a student typing in a chat)
    payload = {
        "user_input": "I'm struggling with calculus derivatives and need practice problems",
        "user_id": "student123"
    }

    # Send the request to our system and get the response
    resp = tc.post("/api/orchestrate_full", json=payload)

    # Print out what the system would tell the student
    print(resp.json())


{'success': True, 'response': 'Generated 5 easy questions on calculus_derivatives', 'intent': 'request_practice_problems', 'topic': 'calculus_derivatives', 'emotional_state': 'frustrated', 'suggestions': ['Generate flashcards for practice', 'Provide concise notes summary', 'Ask for detailed concept explanation'], 'tool_execution': {'tool_name': 'quiz_generator', 'request_params': {'difficulty': 'easy', 'question_type': 'practice', 'num_questions': 5, 'topic': 'calculus_derivatives', 'subject': 'calculus_derivatives'}, 'raw_tool_response': 'Generated 5 easy questions on calculus_derivatives', 'formatted_response': 'Generated 5 easy questions on calculus_derivatives'}, 'session_id': 'student123_1759690584', 'next_actions': ['Review flashcards', 'Request notes', 'Get explanation']}
